In [1]:
#Importing all the packages for the execution of the code
import pandas as pd
import numpy as np
import collections
import itertools

In [2]:
# Defining euclidean distance function and passing the testing and training set that are formed after sampling the data frame formed.


def eucledian_distance(testing_set,training_set):
    distance_euclidean=0     #distance_euclidean will be storing the distance, calculated 
                             #by iterating the training set using the testing set values row-wise that are being sent 
                             #one by one once we call the function.'''   
    for i in range(len(training_set)):
        distance_euclidean=distance_euclidean+np.power((testing_set[i]-training_set[i]),2)
        
    return np.sqrt(distance_euclidean)    #returning the square rooted value of the calculated distance for a partiular row in both the sets 
                                          #back to the part of the code where the function is called.The number of values returned
                                          #will be equal to the number of rows available in the testing set.''' 
    

    

In [3]:
effective_dataframe= pd.read_csv("quality.csv")
#Creating k Random medoids for k=2
k=2
sample_kinstances=effective_dataframe.sample(n=k,replace=False)


dictionary_conewfinal={}
for couter in range(100):
    
    distance_matrix=[]
    dictionary_index_distance={}
    dictionary_final={}
    distance_newclusterhead=[]
    jo =[]
    list_clusters=[]
    lo =[]
    list_newclusterhead=[]
    dictionary_intradistance={}
    dictionary_newfinal={}
    dictionary_index_newdistance={} 
    dictionary_newintradistance={}
    dictionary_new_clusterhead={}
    
    #calculating te euclidean distance between  the random medoids and all the instances in the dataframe and creating a 
    # distance matrix.
    
    for i in range(len(effective_dataframe.values)):
        row_dt=[]
        for j in range(len(sample_kinstances.values)):
            distance=(eucledian_distance(effective_dataframe.values[i],sample_kinstances.values[j]))
            row_dt.append(distance)
        distance_matrix.append(row_dt)
    
    #making a list of tuples in which the i is the index of the instance with minimum distance from the cluster head and the cluster id as cluster head.
    for i in range(len(distance_matrix)):
        distance=min(distance_matrix[i])
        
        cluster_head=distance_matrix[i].index(distance)
        jo.append((i,cluster_head))

        
    #Traversing the list of tuples to finally store the cluster head id as the jkey and vale is the list of instances that belong to
    # that particular cluster.
    for tuplex in jo:
        if tuplex[1] not in dictionary_final.keys():
            dictionary_final[tuplex[1]]=[tuplex[0]]
        else:
            dictionary_final[tuplex[1]].append(tuplex[0])
        
    
    #Finding the data points with minimum cost from all teh datapoints within the cluster and making them the
    #new clusterhead.
    for key,value in dictionary_final.items():
        list_min=[]
        for x in value:
            intra_distance=0
            for j in value:
                intra_distance+=(eucledian_distance(effective_dataframe.iloc[x],effective_dataframe.iloc[j]))
            list_min.append(intra_distance)
        

        min_value=min(list_min)
        index_minvalue=list_min.index(min_value)
        
        dictionary_intradistance[key]=value[index_minvalue]
        list_min=[]
    
    
    df=pd.DataFrame()
    
    #Making a new dataframe of new cluster heads and then again finding the new distances between the new cluster heads and 
    #all teh datapounts to finally create a new cluster, same as done with the random medoids.
    for i in dictionary_intradistance.keys():
        new_clusterhead=(effective_dataframe.iloc[[dictionary_intradistance[i]]])
        df=df.append(new_clusterhead)
    print(df)
    sample_kinstances=df
    
    for i in range(len(effective_dataframe.values)):
        row_dt=[]
        for j in range(len(df.values)):
            distance=(eucledian_distance(effective_dataframe.values[i],df.values[j]))
            row_dt.append(distance)
        distance_newclusterhead.append(row_dt)
        
    
       
    for i in range(len(distance_newclusterhead)):
        distance=min(distance_newclusterhead[i])
        cluster_head=distance_newclusterhead[i].index(distance)
        lo.append((i,cluster_head))

    for tuplex in lo:
        if tuplex[1] not in dictionary_newfinal.keys():
            dictionary_newfinal[tuplex[1]]=[tuplex[0]]
        else:
            dictionary_newfinal[tuplex[1]].append(tuplex[0])
                      
        dictionary_conewfinal.update(dictionary_newfinal)
    #print(dictionary_conewfinal)
            
    #checking if the new cluster formed and the prevoius one are same or not, if yes the loop of creatinf another cluster will 
    #terminate.
    if dictionary_conewfinal==dictionary_final:
        break
    else:
        continue
    


    
                         
            


     InpatientDays  ERVisits  OfficeVisits
112              2         2            22
7                2         0             8
    InpatientDays  ERVisits  OfficeVisits
13              1         1            20
86              1         0             7
    InpatientDays  ERVisits  OfficeVisits
13              1         1            20
86              1         0             7


In [4]:

dict_cc={}
df_pk=effective_dataframe.copy(deep=True)

# Creating a csv file for storing the cluster head alloted to the instances in the dataframe.
for i,value in dictionary_conewfinal.items():
    
    for k in value:
        dict_cc[k]=i
        
df_cluserhead=pd.DataFrame.from_dict(dict_cc,orient='index')

df_clus=df_cluserhead.rename(columns={0:'CLusterhead'})

ff=df_pk.join(df_clus)
ff.to_csv('cluster_2.csv', sep=';', encoding='utf-8')


remaining_list=[]
list_totalpoits=[]
list_sublists=[]
dictioary_B_itradistace={}

# This is used for finding the value of b when k=2, as we just have to find the distance between each point of the cluster and
#the one it does not belong to.

for key,value in dictionary_conewfinal.items():
    for row_idex in value:
        list_totalpoits.append(row_idex)

# This is used for finding the value of b when k=3 and more, as we have to find the distance between each sublist of point of the cluster and
#it does not belong to.
for key,value in dictionary_conewfinal.items():
    list_sublists.append(value)


#For finding the value of a, in which find the average distance between each points and points within the cluster.
for key,value in dictionary_conewfinal.items():
    a_average=[]
    for x in value:
        list_min=[]
        intra_distance=0
        for j in value:
            intra_distance=(eucledian_distance(effective_dataframe.iloc[x],effective_dataframe.iloc[j]))
            list_min.append(intra_distance)
        a_average.append(sum(list_min)/len(list_min))
    dictionary_newintradistance[key]=a_average 

#Finding the value of b by by iterating the list of datapoints of the other clusters and then finding the minimum value of the two values of
#average distance from all the datapoints of other clustrs
if k == 2:
    for key,value in dictionary_conewfinal.items():
        a_average=[]
        remaining_list=[i for i in list_totalpoits if i not in  value]

        for x in value:

            list_min=[]
            intra_distance=0
            for j in remaining_list:
                intra_distance=(eucledian_distance(effective_dataframe.iloc[x],effective_dataframe.iloc[j]))
                list_min.append(intra_distance)

            a_average.append(sum(list_min)/len(list_min))
        
        dictioary_B_itradistace[key]=a_average 
   
    #Using the values of a and b and calculating the average silhoutte width using the known formula of all the points within 
    #cluster. This will give us the average silhoutte width of each cluster and which is then stored in the dictionary
    final={}        
    for key in dictioary_B_itradistace.keys():
        list_eachpoit_width=[]

        for i,j in zip(dictionary_newintradistance[key],dictioary_B_itradistace[key]):

            eachpointwidth=(j-i)/max(i,j)
            list_eachpoit_width.append(eachpointwidth)
        avg_width=(sum(list_eachpoit_width)/len(list_eachpoit_width))
        final[key]=avg_width
    print(final)
    
    #Calculating the average of all the silhoutte widths of each cluster to give a net width value.
    ans=0
    for key, value in final.items():
        ans+=value
    print(ans/len(final),"Average Width")   
            
    
else:
    for key,value in dictionary_conewfinal.items():

        ll=[]
        remaining_list=list_sublists
        remaining_list=[i for i in list_sublists if i != value]
        for x in value:
            list_min=[]
            intra_distance=0
            a_average=[]
            for j in remaining_list:
                for t in j: 
                    intra_distance=(eucledian_distance(effective_dataframe.iloc[x],effective_dataframe.iloc[t]))
                    list_min.append(intra_distance)
                a_average.append(sum(list_min)/len(list_min))
            ll.append(a_average)
        dictioary_B_itradistace[key]=ll

    dict_SS={}
    
     #Using the values of a and b and calculating the average silhoutte width using the known formula of all the points within 
    #cluster. This will give us the average silhoutte width of each cluster and which is then stored in the dictionary.
    for key,value in dictioary_B_itradistace.items():
        jlo=[]
        print(key,value)
        for value1 in value:
            jlo.append(min(value1))

        dict_SS[key]=jlo
    final={}        
    for key in dict_SS.keys():
        list_eachpoit_width=[]

        for i,j in zip(dictionary_newintradistance[key],dict_SS[key]):

            eachpointwidth=(j-i)/max(i,j)
            list_eachpoit_width.append(eachpointwidth)
        avg_width=(sum(list_eachpoit_width)/len(list_eachpoit_width))
        final[key]=avg_width
    print(final)
            

    #Calculating the average of all the silhoutte widths of each cluster to give a net width value.
    ans=0
    for key, value in final.items():
        ans+=value
    print(ans/len(final),"Average Width")   
            



0 [[12.006455050563105], [12.891326280204604], [14.071554253550621], [14.764596215786002], [13.72167839234648], [24.687686759851626], [13.762379908214715], [8.408591183797858], [13.835588786880885], [9.297658683095674], [8.408591183797858], [12.006455050563105], [18.582719067656793], [39.11151546777912], [39.900097978371676], [30.452163999657692], [9.935790114215726], [9.132019989018845], [13.218326366282335], [14.82855208482776], [37.47931806497865], [9.455568206088039], [21.570004970735596], [12.006455050563105], [40.8333376294366], [18.668207786865374], [12.939532008042063], [15.449380741659402], [16.513003090877174], [12.781133988881091], [18.668207786865374], [9.370139655184898], [12.226777609880417], [8.874811771382014], [15.782006838579571], [20.39918592579794], [9.272699809690561], [9.132019989018845], [11.827999198970229], [9.063479595980663], [19.636991101067], [8.831266597704161], [10.172794746110222], [9.278822744792542], [21.470538216686485], [18.655060251332785], [17.6666

In [5]:
#Calculating cluters for k=3
k=3
sample_kinstances=effective_dataframe.sample(n=k,replace=False)


dictionary_conewfinal={}
for couter in range(100):
    
    distance_matrix=[]
    dictionary_index_distance={}
    dictionary_final={}
    distance_newclusterhead=[]
    jo =[]
    list_clusters=[]
    lo =[]
    list_newclusterhead=[]
    dictionary_intradistance={}
    dictionary_newfinal={}
    dictionary_index_newdistance={} 
    dictionary_newintradistance={}
    dictionary_new_clusterhead={}
    
    #calculating te euclidean distance between  the random medoids and all the instances in the dataframe and creating a 
    # distance matrix.
    
    for i in range(len(effective_dataframe.values)):
        row_dt=[]
        for j in range(len(sample_kinstances.values)):
            distance=(eucledian_distance(effective_dataframe.values[i],sample_kinstances.values[j]))
            row_dt.append(distance)
        distance_matrix.append(row_dt)
    
    #making a list of tuples in which the i is the index of the instance with minimum distance from the cluster head and the cluster id as cluster head.
    for i in range(len(distance_matrix)):
        distance=min(distance_matrix[i])
        
        cluster_head=distance_matrix[i].index(distance)
        jo.append((i,cluster_head))

        
    #Traversing the list of tuples to finally store the cluster head id as the jkey and vale is the list of instances that belong to
    # that particular cluster.
    for tuplex in jo:
        if tuplex[1] not in dictionary_final.keys():
            dictionary_final[tuplex[1]]=[tuplex[0]]
        else:
            dictionary_final[tuplex[1]].append(tuplex[0])
        
    
    #Finding the data points with minimum cost from all teh datapoints within the cluster and making them the
    #new clusterhead.
    for key,value in dictionary_final.items():
        list_min=[]
        for x in value:
            intra_distance=0
            for j in value:
                intra_distance+=(eucledian_distance(effective_dataframe.iloc[x],effective_dataframe.iloc[j]))
            list_min.append(intra_distance)
        

        min_value=min(list_min)
        index_minvalue=list_min.index(min_value)
        
        dictionary_intradistance[key]=value[index_minvalue]
        list_min=[]
    
    
    df=pd.DataFrame()
    
    #Making a new dataframe of new cluster heads and then again finding the new distances between the new cluster heads and 
    #all teh datapounts to finally create a new cluster, same as done with the random medoids.
    for i in dictionary_intradistance.keys():
        new_clusterhead=(effective_dataframe.iloc[[dictionary_intradistance[i]]])
        df=df.append(new_clusterhead)
    print(df)
    sample_kinstances=df
    
    for i in range(len(effective_dataframe.values)):
        row_dt=[]
        for j in range(len(df.values)):
            distance=(eucledian_distance(effective_dataframe.values[i],df.values[j]))
            row_dt.append(distance)
        distance_newclusterhead.append(row_dt)
        
    
       
    for i in range(len(distance_newclusterhead)):
        distance=min(distance_newclusterhead[i])
        cluster_head=distance_newclusterhead[i].index(distance)
        lo.append((i,cluster_head))

    for tuplex in lo:
        if tuplex[1] not in dictionary_newfinal.keys():
            dictionary_newfinal[tuplex[1]]=[tuplex[0]]
        else:
            dictionary_newfinal[tuplex[1]].append(tuplex[0])
                      
        dictionary_conewfinal.update(dictionary_newfinal)
    #print(dictionary_conewfinal)
            
    #checking if the new cluster formed and the prevoius one are same or not, if yes the loop of creatinf another cluster will 
    #terminate.
    if dictionary_conewfinal==dictionary_final:
        break
    else:
        continue
    


    
                         
            


     InpatientDays  ERVisits  OfficeVisits
101              1         2            11
4                8         2            19
76              10         1             3
     InpatientDays  ERVisits  OfficeVisits
41               1         0             9
112              2         2            22
23               8         4             6
     InpatientDays  ERVisits  OfficeVisits
112              2         2            22
15               0         0             8
77               9         4             6
    InpatientDays  ERVisits  OfficeVisits
13              1         1            20
15              0         0             8
77              9         4             6
    InpatientDays  ERVisits  OfficeVisits
13              1         1            20
86              1         0             7
77              9         4             6
    InpatientDays  ERVisits  OfficeVisits
13              1         1            20
86              1         0             7
77              9     

In [6]:
dict_cc={}
df_pk=effective_dataframe.copy(deep=True)

# Creating a csv file for storing the cluster head alloted to the instances in the dataframe.
for i,value in dictionary_conewfinal.items():
    
    for k in value:
        dict_cc[k]=i
        
df_cluserhead=pd.DataFrame.from_dict(dict_cc,orient='index')

df_clus=df_cluserhead.rename(columns={0:'CLusterhead'})

ff=df_pk.join(df_clus)
ff.to_csv('cluster_3.csv', sep=';', encoding='utf-8')


remaining_list=[]
list_totalpoits=[]
list_sublists=[]
dictioary_B_itradistace={}

# This is used for finding the value of b when k=2, as we just have to find the distance between each point of the cluster and
#the one it does not belong to.

for key,value in dictionary_conewfinal.items():
    for row_idex in value:
        list_totalpoits.append(row_idex)

# This is used for finding the value of b when k=3 and more, as we have to find the distance between each sublist of point of the cluster and
#it does not belong to.
for key,value in dictionary_conewfinal.items():
    list_sublists.append(value)


#For finding the value of a, in which find the average distance between each points and points within the cluster.
for key,value in dictionary_conewfinal.items():
    a_average=[]
    for x in value:
        list_min=[]
        intra_distance=0
        for j in value:
            intra_distance=(eucledian_distance(effective_dataframe.iloc[x],effective_dataframe.iloc[j]))
            list_min.append(intra_distance)
        a_average.append(sum(list_min)/len(list_min))
    dictionary_newintradistance[key]=a_average 

#Finding the value of b by by iterating the list of datapoints of the other clusters and then finding the minimum value of the two values of
#average distance from all the datapoints of other clustrs
if k == 2:
    for key,value in dictionary_conewfinal.items():
        a_average=[]
        remaining_list=[i for i in list_totalpoits if i not in  value]

        for x in value:

            list_min=[]
            intra_distance=0
            for j in remaining_list:
                intra_distance=(eucledian_distance(effective_dataframe.iloc[x],effective_dataframe.iloc[j]))
                list_min.append(intra_distance)

            a_average.append(sum(list_min)/len(list_min))
        
        dictioary_B_itradistace[key]=a_average 
   
    #Using the values of a and b and calculating the average silhoutte width using the known formula of all the points within 
    #cluster. This will give us the average silhoutte width of each cluster and which is then stored in the dictionary
    final={}        
    for key in dictioary_B_itradistace.keys():
        list_eachpoit_width=[]

        for i,j in zip(dictionary_newintradistance[key],dictioary_B_itradistace[key]):

            eachpointwidth=(j-i)/max(i,j)
            list_eachpoit_width.append(eachpointwidth)
        avg_width=(sum(list_eachpoit_width)/len(list_eachpoit_width))
        final[key]=avg_width
    print(final)
    
    #Calculating the average of all the silhoutte widths of each cluster to give a net width value.
    ans=0
    for key, value in final.items():
        ans+=value
    print(ans/len(final),"Average Width")   
            
else:
    for key,value in dictionary_conewfinal.items():

        ll=[]
        remaining_list=list_sublists
        remaining_list=[i for i in list_sublists if i != value]
        for x in value:
            list_min=[]
            intra_distance=0
            a_average=[]
            for j in remaining_list:
                for t in j: 
                    intra_distance=(eucledian_distance(effective_dataframe.iloc[x],effective_dataframe.iloc[t]))
                    list_min.append(intra_distance)
                a_average.append(sum(list_min)/len(list_min))
            ll.append(a_average)
        dictioary_B_itradistace[key]=ll

    dict_SS={}
    
     #Using the values of a and b and calculating the average silhoutte width using the known formula of all the points within 
    #cluster. This will give us the average silhoutte width of each cluster and which is then stored in the dictionary.
    for key,value in dictioary_B_itradistace.items():
        jlo=[]
        print(key,value)
        for value1 in value:
            jlo.append(min(value1))

        dict_SS[key]=jlo
    final={}        
    for key in dict_SS.keys():
        list_eachpoit_width=[]

        for i,j in zip(dictionary_newintradistance[key],dict_SS[key]):

            eachpointwidth=(j-i)/max(i,j)
            list_eachpoit_width.append(eachpointwidth)
        avg_width=(sum(list_eachpoit_width)/len(list_eachpoit_width))
        final[key]=avg_width
    print(final)
    
    #Calculating the average of all the silhoutte widths of each cluster to give a net width value.
    ans=0
    for key, value in final.items():
        ans+=value
    print(ans/len(final),"Average Width")   
            

0 [[11.260023845195153, 12.265964548002591], [12.20211151125715, 13.121851553111437], [14.218632649537188, 14.072646388600504], [14.757040160080116, 14.786520541878373], [13.153495655810167, 13.906881372883225], [24.646864375934644, 24.58274140333954], [13.360982373825562, 13.904388417585508], [7.449685663838789, 8.776946366051671], [13.184226520793484, 14.042655619079701], [9.12112115753356, 9.504216316451812], [7.449685663838789, 8.776946366051671], [11.260023845195153, 12.265964548002591], [19.17219269610277, 18.37969351813497], [39.255182435813, 38.85554506364241], [39.70714957725257, 39.77772196271145], [30.15005755722505, 30.402324779475006], [9.41121307468072, 10.19253978666566], [8.334773992444658, 9.451416840013723], [12.978098887625302, 13.328754108207509], [14.197180248579311, 15.017712751481803], [38.517399060892835, 36.9442412808412], [8.633015665142281, 9.777798808125297], [21.13828110797167, 21.62570892340398], [11.260023845195153, 12.265964548002591], [41.06795011099859